In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
from scipy.spatial.distance import euclidean
pd.options.mode.chained_assignment = None

# Motivation

**I mostly wanted to get familiar with the tracking data, as it's my first time of being exposed to this type of data. I hope you still find some value in this notebook.**

In [ ]:
tracking2020 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')

In [ ]:
tracking2020.info()

In [ ]:
# only looking at data from plays when the home team is on the offense
right = tracking2020[tracking2020['playDirection'] == 'right']
# only looking at a specific match
match = right[right['gameId'] == 2020091312]

# Heatmap Tackles

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
plt.hist2d(right['x'][right['event'] == 'tackle'], right['y'][right['event'] == 'tackle'],bins=70, cmap='magma')
plt.xlim(0 , 120)
plt.ylim(0,  53.3)
plt.title('Heatmap of all player locations during a tackle when the offense is moving to the right')
plt.show()

# Scatterplots and KDE-Plots for a Specific Play

In [ ]:
pal = ['darkturquoise', 'deeppink', 'black']
fig, axs = plt.subplots(2, 2, figsize=(20,10))
sns.scatterplot(ax=axs[0,0], data=match[match['playId'] == 2382], x='x', y='y', hue='team', palette=pal)
sns.scatterplot(ax=axs[0,1], data=match[match['playId'] ==  3986], x='x', y='y', hue='team', palette=pal)
sns.kdeplot(ax=axs[1,0], data=match[match['playId'] == 2382], x='x', hue='team', palette=pal)
sns.kdeplot(ax=axs[1,1], data=match[match['playId'] ==  3986], x='x',hue='team', palette=pal)
plt.show()

# Looking at Taylor Rapp's Movements during Play 2382 of Game 2020091312

In [ ]:
# looking at a specific player during play 2382 'Taylor Rapp'
player = match[match['nflId'] == 47844.0]
player = player[player['playId'] ==  2382]
ball = match[match['team'] ==  'football']

In [ ]:
# 90° window around 0°&360° 
def change_in_angle(angles):
    last = angles[0]
    change = []
    for a in angles:
        change.append(a - last)
        if a <45 and last > 315:
            change[-1] += 360
        elif a > 315 and last < 45:
            change[-1] -= 360
        last = a
    return change

# euclidean distance between player and ball at each moment
def dist_to_ball(ball_df, player_df):
    dist = []
    for b, p in zip(ball_df.index, player_df.index):
        dist.append(euclidean([ball_df['x'][b], ball_df['y'][b]],[player_df['x'][p], player_df['y'][p]]))
    return dist

In [ ]:
player['dir_change'] = change_in_angle(player['dir'][player['playId'] ==  2382].to_list())
player['o_change'] = change_in_angle(player['o'][player['playId'] ==  2382].to_list())
player['dtb'] = dist_to_ball(ball[ball['playId'] == 2382], player)

In [ ]:
player.head()

In [ ]:
player['time'] =  pd.to_datetime(player['time'])
ball['time'] =  pd.to_datetime(ball['time'])
fig, axs = plt.subplots(2, 2, figsize=(20,10))
sns.scatterplot(ax=axs[0,0], data=player, x='x', y='y', hue='dtb', palette='cool')
sns.scatterplot(ax=axs[0,0], data=ball[ball['playId'] ==  2382], x='x', y='y', color='black')
axs[0,0].set_title('Player Movement - Ball Movement')
axs[0,1].plot(player['time'], player['dir_change'], color = 'darkturquoise')
axs[0,1].plot(player['time'], player['o_change'], color = 'deeppink')
axs[0,1].axhline(y=0.5, color='black', linestyle='-')
axs[0,1].set_title('Change in Direction/Orientation')
axs[0,1].legend(('Change in Direction','Change in Orientation'))
axs[1,0].plot(player['time'], player['s'], color = 'darkturquoise')
axs[1,0].plot(ball['time'][ball['playId'] == 2382], ball['s'][ball['playId'] == 2382], color = 'deeppink')
axs[1,0].set_title('Player/Ball Speed')
axs[1,0].legend(('Player','Ball'))
axs[1,1].plot(player['time'], player['dtb'], color = 'deeppink')
axs[1,1].set_title('Distance to the Ball')
plt.show()

Our player starts moving and shifting his direction before the ball is moving. The player's direction is mostly steady with a slight tendency to the right as the ball is picking up speed. He is accelerating with no sudden change in direction unlike his orientation. His distance to the ball is increasing. The ball is losing speed until it is picked up by an opposing player. The distance to the ball is decreasing again as the opposing player is running towards him. We see the orientation of our player move to the left shortly before his direction shifts abruptly to the left. The distance between our player, the opposing player and the ball decreases further.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.plot(player['time'], list(np.cumsum(abs(player['o_change']))), color = 'deeppink')
ax1.plot(player['time'], list(np.cumsum(abs(player['dir_change']))), color = 'darkturquoise')
ax1.set_title('Total Change in Direction/Orientation')
ax1.legend(('Orientation', 'Direction'))
ax2.plot(player['time'], list(np.cumsum(player['dis'])), color = 'deeppink')
ax2.set_title('Total Distance Traveled')
plt.show()